In [1]:
import os; import sys; import re

# common spark import
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType

# connect to spark if we haven't already
if not 'spark' in locals():
  spark = SparkSession.builder \
      .master("local[*]") \
      .appName('development') \
      .config("spark.sql.debug.maxToStringFields", str(1024 * 1024)) \
      .getOrCreate()
  sc = spark.sparkContext

print("Connected to Spark!")

Connected to Spark!


In [4]:
df = spark.read.option("header", "true").csv("address-linkage-key/address_link/data/test/*small*.gz")
df.limit(10).toPandas()

,predirection,streetname,streetsuffix,postdirection,unitdesignator,unitdesignatornumber,cityname,state,zipcode,zip_4,dwellingtype,address_line_1,address_line_2,expected_dpc,expected_check_digit
0,S,Main,St,,,,Aspers,PA,17304,9802,S,259 S Main St,None,59,2
1,,Joseph,Ave,,,,Westfield,MA,01085,1812,S,75 Joseph Ave,None,75,2
2,,Algonquin,Way,,,,Orangevale,CA,95662,2342,S,8751 Algonquin Way,None,51,5
3,N,Main,St,,Apt,2,Crown Point,IN,46307,3279,M,250 N Main St,Apt 2,02,7
4,,252nd,St,,,,Harbor City,CA,90710,2422,S,1136 252nd St,None,36,4
5,,Agean,Ct,,,,Murrieta,CA,92562,2118,S,41484 Agean Ct,None,84,2
6,W,43rd,St,,Apt,25E,New York,NY,10036,6473,M,350 W 43rd St,Apt 25E,55,0
7,,Muirfield,Run,,,,Norcross,GA,30093,6100,S,7302 Muirfield Run,None,02,6
8,NW,37th,St,,,,Coral Springs,FL,33065,2782,S,11075 NW 37th St,None,75,2
9,N,Point,Blvd,,Apt,903,Tallahassee,FL,32308,4181,M,1950 N Point Blvd,Apt 903,28,0


In [5]:
df.printSchema()

root
 |-- predirection: string (nullable = true)
 |-- streetname: string (nullable = true)
 |-- streetsuffix: string (nullable = true)
 |-- postdirection: string (nullable = true)
 |-- unitdesignator: string (nullable = true)
 |-- unitdesignatornumber: string (nullable = true)
 |-- cityname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- zip_4: string (nullable = true)
 |-- dwellingtype: string (nullable = true)
 |-- address_line_1: string (nullable = true)
 |-- address_line_2: string (nullable = true)
 |-- expected_dpc: string (nullable = true)
 |-- expected_check_digit: string (nullable = true)



In [6]:
#add dpc column to dataframe
df = df.withColumn('dpc', f.lit(None))

In [7]:
df.printSchema()

root
 |-- predirection: string (nullable = true)
 |-- streetname: string (nullable = true)
 |-- streetsuffix: string (nullable = true)
 |-- postdirection: string (nullable = true)
 |-- unitdesignator: string (nullable = true)
 |-- unitdesignatornumber: string (nullable = true)
 |-- cityname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- zip_4: string (nullable = true)
 |-- dwellingtype: string (nullable = true)
 |-- address_line_1: string (nullable = true)
 |-- address_line_2: string (nullable = true)
 |-- expected_dpc: string (nullable = true)
 |-- expected_check_digit: string (nullable = true)
 |-- dpc: null (nullable = true)



In [9]:
from pyspark.sql.functions import col
import re

In [10]:
##df = df.filter(df.dwellingtype == "S")

In [13]:
df.count()

1000

In [16]:
'''
def get_dpc(address_line):
    #split address line string elements into a list
    address_split = address_line.split()

    #use regex to match and then return the last two values in the specified element
    #returns no value if address line does not match pattern
    if re.match(r'^\d{2}', address_line):
        res = address_split[0]
        return res[-2] + res[-1]
    elif re.match(r'PO BOX \d{2,5}', address_line):
        res = address_split[2]
        return res[-2] + res[-1]
    elif re.match(r'RR \d+ BOX \d{2,5}', address_line):
        res = address_split[3]
        return res[-2] + res[-1]
    elif re.match(r'HC \d+ BOX \d{2,5}', address_line):
        res = address_split[3]
        return res[-2] + res[-1]
    else:
        return
'''

"\ndef get_dpc(address_line):\n    #split address line string elements into a list\n    address_split = address_line.split()\n\n    #use regex to match and then return the last two values in the specified element\n    #returns no value if address line does not match pattern\n    if re.match(r'^\\d{2}', address_line):\n        res = address_split[0]\n        return res[-2] + res[-1]\n    elif re.match(r'PO BOX \\d{2,5}', address_line):\n        res = address_split[2]\n        return res[-2] + res[-1]\n    elif re.match(r'RR \\d+ BOX \\d{2,5}', address_line):\n        res = address_split[3]\n        return res[-2] + res[-1]\n    elif re.match(r'HC \\d+ BOX \\d{2,5}', address_line):\n        res = address_split[3]\n        return res[-2] + res[-1]\n    else:\n        return\n"

In [15]:
#get_dpc('1234 MAIN ST')

,predirection,streetname,streetsuffix,postdirection,unitdesignator,unitdesignatornumber,cityname,state,zipcode,zip_4,dwellingtype,address_line_1,address_line_2,expected_dpc,expected_check_digit,dpc
0,,Circle,Dr,,,,Fort Morgan,CO,80701,3419,S,525 Circle Dr,None,25,0,None
1,,Primrose,Ave,,,,Vista,CA,92083,8032,S,2312 Primrose Ave,None,12,2,None
2,,Gardenstone,Cir,,,,Tallmadge,OH,44278,1085,S,849 Gardenstone Cir,None,49,8,None
3,,Briarwood,Dr,,,,Crestwood,KY,40014,9019,S,7511 Briarwood Dr,None,11,0,None
4,S,15th,St,,,,Saint Clair,MI,48079,5203,S,1071 S 15th St,None,71,4,None
5,,Front,St,,Apt,4B,Brooklyn,NY,11201,1223,M,206 Front St,Apt 4B,42,1,None
6,,Millpond,Rd,,,,Elizabeth Cty,NC,27909,7551,S,1295 Millpond Rd,None,95,1,None
7,,Glenwood,Ln,,,,East Meadow,NY,11554,3719,S,479 Glenwood Ln,None,79,8,None
8,NW,9th,St,,Apt,105,Miami,FL,33125,3443,M,2150 NW 9th St,Apt 105,30,9,None
9,,10th,St,NE,,,Naples,FL,34120,2057,S,460 10th St NE,None,60,0,None


In [12]:
#convert python function into PySpark SQL UDF
#returns a string
#getdpcUDF = f.udf(lambda z: get_dpc(z),StringType())

In [13]:
#df = df.withColumn('dpc', getdpcUDF(col('address_line_1')))

In [14]:
#df.limit(10).toPandas()

In [15]:
#dpc = dpcUDF(col('address_line_1'))

#create new dataframe with calculated dpc
#uses when function to exclude rows that where dwelling type is equal to M

#df2 = df.withColumn('dpc',
                    #f.when(
                        #f.col('dwellingtype') != "M", 
                        #getdpcUDF(col('address_line_1'))
                    #).otherwise(f.col('dpc'))
                   #)

In [27]:
df4 = df.withColumn('dpc', (f.regexp_extract(f.col('address_line_1'),'((?<!\S)\d+(?!\S))', 1)).substr(-2,2))
df4.limit(100).toPandas() 

AnalysisException: Can't extract value from address_line_1#27: need struct type but got string;

In [ ]:
"""
test_values = ['1234 MAIN ST', 'PO BOX 4444', 'RR 12 BOX 154', 'HC 1 BOX 1264', 
               '4 Main St', 'Main St', 'RR 12', 'RR 1 BOX 123/124/12']

for i in range(len(test_values)):
    address_line = test_values[i]
    address_split = address_line.split()
    print('Address Line 1: ' + address_line)

    if re.match(r'^\d{2}', address_line):
        res = address_split[0]
        print('DPBC: ' + res[-2] + res[-1])
    elif re.match(r'PO BOX \d{2,5}', address_line):
        res = address_split[2]
        print('DPBC: ' + res[-2] + res[-1])
    elif re.match(r'RR \d+ BOX \d{2,5}', address_line):
        res = address_split[3]
        print('DPBC: ' + res[-2] + res[-1])
    elif re.match(r'HC \d+ BOX \d{2,5}', address_line):
        res = address_split[3]
        print('DPBC: ' + res[-2] + res[-1])
    else:
        print('exception not accounted for')
"""